# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_path='../cities.csv'
file =pd.read_csv(file_path)
df = pd.DataFrame(file)
df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#use  jupyter-gmaps and the Google Places API 
#Note: if you having trouble displaying the maps, try running jupyter nbextension enable --py gmaps in your environment and retry.

In [4]:
gmaps.configure(api_key = g_key)

coordinates = df[['Lat', 'Lng']]
humidity = df['Humidity']

figure_layout = {
    'width': '800px',
    'height': '600px',
    'border':'1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
max_intensity = np.max(humidity)
fig = gmaps.figure(layout = figure_layout)
heat_layer = gmaps.heatmap_layer(coordinates, weights = humidity, dissipating = False, max_intensity = max_intensity, point_radius = 4)

fig.add_layer(heat_layer)
fig 

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_weather = df.loc[(df['Max Temp']< 80) & (df['Max Temp']> 70) & (df['Wind Speed']< 10) & (df['Cloudiness']<10)].dropna()
ideal_weather.head(30)

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
268,268,dingle,4,PH,1558378836,80,11.00,122.67,79.08,2.73
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
402,402,sambava,7,MG,1558378871,99,-14.27,50.17,70.98,1.50
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df = pd.DataFrame()
hotel_df['Hotel Name'] = ''

hotel_params = {
    "type": 'hotel',
    "radius": 5000,
    'key': g_key   
}
place_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for  index, row in ideal_weather.iterrows():
    hotel_params['location'] = f"{row['Lat']}, {row['Lng']}"
    
    results = requests.get(place_url, params=hotel_params).json()
    hotel_df.loc[index, 'Hotel Name']=results['results'][1]['name']
hotel_df= pd.merge(hotel_df, ideal_weather, left_index=True, right_index=True)
hotel_df

,Hotel Name,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,Hotel Escarpas do Lago,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
87,Agence BNI Madagascar,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52
88,Conselho Tutelar,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,La Case Bambou,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,Dandeli Lake County,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,Hôtel La Giralda,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
268,Rufino G. Palabrica Sr. National High School,268,dingle,4,PH,1558378836,80,11.00,122.67,79.08,2.73
297,MOGOK HILL HOTEL,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
402,DHL Express,402,sambava,7,MG,1558378871,99,-14.27,50.17,70.98,1.50
444,Maskan Bank,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,
                            info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…